In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
# nltk.download()
# Only download stopwords!
stop_words = nltk.stopwords

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', None)

AttributeError: module 'nltk' has no attribute 'stopwords'

In [ ]:
wine = pd.read_csv('redwine.csv', delimiter=';')
chemColNames = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']
chem = wine[chemColNames]

colErrorPairs = {
    'density'    : [' . '],
    'citric acid': [' - ',' -   '],
    'alcohol'    : ['100.333.333.333.333','11.066.666.666.666.600','956.666.666.666.667','923.333.333.333.333']}

for colName in colErrorPairs:
    for faultyString in colErrorPairs[colName]:
        wine[colName] = wine[colName].replace(faultyString,np.nan)
        
wine['alcohol'] = wine['alcohol'].astype(float)
wine['density'] = wine['density'].astype(float)
wine['citric acid'] = wine['citric acid'].astype(float)

# wine['points'] = (wine['points'] - wine['points'].mean())/wine['points'].std(ddof=0)

# Onderzoeksvraag 3: Kun je op basis van bepaalde keywords in de beschrijving een voorspelling doen over hoe hoog de score van deze wijn is? (Supervised - Regression/ Datavisualisatie)

We willen onderzoeken of je de score van de wijn kunt bepalen aan de hand van sommige keywords die gevonden zijn in de titels en de beschrijving.

In [ ]:
wordsPerWine = wine[['title','description']]
allWordsSeperated = np.array(' '.join(wordsPerWine.to_numpy().flatten()).replace(',','').split(" "))

In [ ]:
allWordsSeperated = np.char.lower(allWordsSeperated)
(allWordsSeperated[:20],'Amount of words',allWordsSeperated.shape)

In [ ]:
unique, counts = np.array(np.unique(allWordsSeperated, return_counts=True))
def initWordFrame():
    wordCount = np.array([np.array(unique), np.array(counts)])
    wordFrame = pd.DataFrame(wordCount[0],wordCount[1]).reset_index()
    wordFrame.columns = ['count','word']
    wordFrame['count'] = wordFrame['count'].astype(int)
    return wordFrame
wordFrame = initWordFrame()

In [ ]:
print(f'Total of unique seperated words: {wordFrame.shape[0]}')
minCount = 400
# print(wordFrame['count'])
wordFrame.drop(wordFrame[ (wordFrame['word'].isin(stop_words)) | (wordFrame['count'] < minCount)].index , inplace=True)
print(f'Total of unique seperated words with atleast a count of {minCount} & not a stopword: {wordFrame.shape[0]}')
# wordFrame.sort_values(by=['count'], ascending=False).reset_index(drop=True)[:30]

In [ ]:
wordFrame.boxplot(column='count')

In [ ]:
# Make a new column where we put a count of how many keywords are in the description and or title
wine['keyword_count'] = wine['description'].str.count('|'.join(wordFrame['word'].to_list()))
wine[['points', 'keyword_count','title','description']]

In [ ]:
wine[wine['points'].isnull()]

In [ ]:
features = ['keyword_count','price']

# Feature waardes
X = wine[features]
X.fillna(X.mean(),inplace=True)

# Target waardes
y = wine['points']
y.fillna(y.mean(),inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6)

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
score = linreg.score(X_test,y_test)
yPred = linreg.predict(X_test)
meanError = np.sqrt(mean_squared_error(y_test,yPred))
print(f'Score: {score}, Mean Error: {meanError}')

In [ ]:
maxRange = 18
minRange = 3
maxNumberRanges = [x*x for x in reversed(range(minRange,maxRange))]

# Target waardes
y = wine['points']
y.fillna(y.mean(),inplace=True)

for minCount in maxNumberRanges:
    
    wordFrame = initWordFrame()
    wordFrame.drop(wordFrame[ (wordFrame['word'].isin(stop_words)) | (wordFrame['count'] < minCount)].index , inplace=True)
#     Veranderd niks aan de score
#     wordFrame['count'] = (wordFrame['count'] - wordFrame['count'].mean())/wordFrame['count'].std(ddof=0)

    print(f'Total of unique seperated words with atleast a count of {minCount} & not a stopword: {wordFrame.shape[0]}')
    
    wine['keyword_count'] = wine['description'].str.count('|'.join(wordFrame['word'].to_list()))
    
    # Feature waardes
    X = wine[features]
    X.fillna(X.mean(),inplace=True)

    

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6)
    
    linRegTrial = LinearRegression()
    linRegTrial.fit(X_train, y_train)
    score = linRegTrial.score(X_test,y_test)
    yPred = linRegTrial.predict(X_test)
    meanError = np.sqrt(mean_squared_error(y_test,y_pred))
    print(f'Score: {score}, Mean Error: {meanError}')

In [ ]:
score